In [1]:
pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 19.5 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.9 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=1583be64934b8cc97cf85589dfbcf5f76666e017f01ae27986d056ae000550ca
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import numpy as np
import pandas as pd
import faiss
from sentence_transformers import SentenceTransformer
import google.generativeai as genai
import uuid  # For generating UUIDs

############################################
# 1) Read data from CSV file
############################################
CSV_FILE_PATH = "sample.csv"  # Replace with your actual CSV file path

# Load the CSV file into a Pandas DataFrame
# Assuming the CSV has a column named "context" with text data to process
df = pd.read_csv(CSV_FILE_PATH)
print(f"Loaded {len(df)} rows from CSV file.")

############################################
# 2) Process the DataFrame
############################################
# Create chunks of text from the "context" column
records = []
MAX_TOKENS = 3000  # Example token limit for chunking

for _, row in df.iterrows():
    # Safely get the context field
    context = str(row.get("context", "")).strip()  # Ensure the column name matches your CSV
    if context:
        # Create chunks by token size if necessary
        words = context.split()
        chunks = [" ".join(words)]  # Modify as needed to split into smaller chunks
        for chunk in chunks:
            records.append({
                "chunk": chunk,
                "uuid": str(uuid.uuid4())  # Assign a unique UUID to each chunk
            })

# Convert to a Pandas DataFrame
processed_df = pd.DataFrame(records)
print(f"Created {len(processed_df)} rows after chunking.")

# Replace NaN values with empty strings
processed_df.fillna("", inplace=True)

############################################
# 3) Save the processed data to CSV
############################################
output_csv_path = "processed_data.csv"
processed_df.to_csv(output_csv_path, index=False)
print(f"Processed data saved to {output_csv_path}.")


Loaded 100 rows from CSV file.
Created 100 rows after chunking.
Processed data saved to processed_data.csv.


In [5]:
processed_df.head()

,chunk,uuid
0,"Brand Name: Bismuth Stibium, Generic Name: BIS...",0a8f5bf0-029b-4118-bb04-a386cc3277b2
1,"Effective Time: 20240118, Effective Date: 2024...",df27e863-c982-483f-a595-ea35184a613d
2,"Brand Name: Omeprazole, Sodium bicarbonate, Ge...",3db0e559-ef0a-4495-87fc-0e0470183f67
3,Brand Name: Diltiazem Hydrochloride Extended-R...,fb29119a-fd90-4251-94dc-a60f2081a4e5
4,"Brand Name: FLUDROCORTISONE ACETATE, Generic N...",12049f60-5295-45cd-83f2-aba20e891ba5


In [6]:
processed_df.iloc[0, 0]

'Brand Name: Bismuth Stibium, Generic Name: BISMUTH STIBIUM, Manufacturer Name: Uriel Pharmacy Inc., Product Type: HUMAN OTC DRUG, Route: TOPICAL, Effective Time: 20240122, Effective Date: 2024-01-22, Active Ingredient: Active Ingredients: 100 gm contains: 25 gm Allium sativa (Garlic) 1X, 25 gm Chelidonium (Greater celandine) 1X, 25 gm Curcuma (Turmeric) 1X, 25 gm Thuja (American arborvitae) 1X, 20 gm Bismuth 2X, 20 gm Stibium met. (Antimony) 2X, Warnings: Warnings: FOR EXTERNAL USE ONLY. Claims based on traditional homeopathic practice, not accepted medical evidence. Not FDA evaluated. Do not use if allergic to any ingredient. Consult a doctor before use for serious conditions, if conditions worsen or persist, or accidental ingestion occurs. If pregnant or nursing, consult a doctor before use. Avoid contact with eyes. Do not use if safety seal is broken or missing., Indications and Usage: Directions: FOR TOPICAL USE ONLY., Purpose: Use: Temporary relief of warts., Dosage and Administr

In [7]:
from sentence_transformers import SentenceTransformer

# Load the SentenceTransformer model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Generate embeddings for the context column
processed_df['embeddings'] = processed_df['chunk'].apply(lambda x: model.encode(x).tolist())

# Save embeddings for later use
processed_df.to_parquet("data_with_embeddings_final.parquet", index=False)
print("Data with embeddings saved successfully!")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Data with embeddings saved successfully!


In [8]:
import faiss
import numpy as np

# Convert embeddings to a numpy array
embeddings = np.array(processed_df['embeddings'].tolist(), dtype='float32')

# Create a FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# Save the FAISS index
faiss.write_index(index, "faiss_index_final.bin")
print("FAISS index saved successfully!")


FAISS index saved successfully!


In [9]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

def generate_answer_with_context_t5(query, context):
    model_name = "t5-small"  # or "t5-base", "t5-large"
    model = T5ForConditionalGeneration.from_pretrained(model_name)
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    
    input_text = f"Context: {context}\nQuestion: {query}\nAnswer:"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    
    outputs = model.generate(input_ids=inputs['input_ids'], max_length=100, num_beams=4, early_stopping=True)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text


In [10]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

def generate_answer_with_context_gpt2(query, context):
    model_name = "gpt2"  # You can use "gpt2-medium", "gpt2-large" for larger models
    model = GPT2LMHeadModel.from_pretrained(model_name)
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    
    input_text = f"Context: {context}\nQuestion: {query}\nAnswer:"
    
    # Increase max_length to a higher value, ensuring it's within the model's token limit
    max_input_length = 1024  # Ensure this is within the model's max token limit
    inputs = tokenizer(input_text, return_tensors="pt", max_length=max_input_length, truncation=True)
    
    # Set max_new_tokens to control the length of the generated answer
    outputs = model.generate(input_ids=inputs['input_ids'], max_new_tokens=150, num_return_sequences=1, no_repeat_ngram_size=2, temperature=0.7)
    
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Remove the prompt part from the output and return the answer
    answer = generated_text[len(input_text):].strip()
    return answer


In [11]:
from transformers import BertForQuestionAnswering, BertTokenizer
import torch


def generate_answer_with_context_bert(query, context):
    model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
    model = BertForQuestionAnswering.from_pretrained(model_name)
    tokenizer = BertTokenizer.from_pretrained(model_name)

    # Prepare the input text for BERT
    input_text = f"Context: {context}\nQuestion: {query}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    
    # Print the tokenized input to check if the context and question are properly encoded
    # print("Tokenized Input:", inputs)

    # Run the model to get start and end logits
    outputs = model(**inputs)
    start_scores, end_scores = outputs.start_logits, outputs.end_logits

    # Print the logits to see what the model is predicting
    # print("Start Scores:", start_scores)
    # print("End Scores:", end_scores)

    # Get the most likely start and end positions
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores) + 1

    # Check if the answer span is valid
    if answer_start >= answer_end:
        print("No valid answer found.")
        return "No valid answer found."

    # Decode the answer (removes [CLS], [SEP] tokens and gives you the answer span)
    answer = tokenizer.decode(inputs.input_ids[0][answer_start:answer_end], skip_special_tokens=True)
    
    return answer


In [12]:
from transformers import DistilBertForQuestionAnswering, DistilBertTokenizer
import torch

def generate_answer_with_context_distilbert(query, context):
    model_name = "distilbert-base-uncased-distilled-squad"
    model = DistilBertForQuestionAnswering.from_pretrained(model_name)
    tokenizer = DistilBertTokenizer.from_pretrained(model_name)

    input_text = f"Context: {context}\nQuestion: {query}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

    outputs = model(**inputs)
    start_scores, end_scores = outputs.start_logits, outputs.end_logits
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores) + 1
    answer = tokenizer.decode(inputs.input_ids[0][answer_start:answer_end])
    return answer


In [13]:
from transformers import BloomForCausalLM, BloomTokenizerFast

def generate_answer_with_context_bloom(query, context):
    model_name = "bigscience/bloom-560m"
    model = BloomForCausalLM.from_pretrained(model_name)
    tokenizer = BloomTokenizerFast.from_pretrained(model_name)

    # Create the prompt
    input_text = f"Given the following context:\n{context}\n\nQuestion: {query}\nAnswer:"
    
    # Tokenize input text
    inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True, padding=True)

    # Generate the response
    outputs = model.generate(input_ids=inputs['input_ids'], max_new_tokens=100)  # Control only new token generation

    # Decode the generated tokens to text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_text = generated_text.split('Question: ')[-1]

    return generated_text


In [14]:

# Load the FAISS index
index = faiss.read_index("faiss_index_final.bin")

# Example query
query = "Bismuth Stibium what is used from ?"
query_embedding = model.encode(query).astype('float32').reshape(1, -1)

# Retrieve the top 5 most relevant contexts
distances, indices = index.search(query_embedding, 5)

# Extract the relevant contexts
relevant_contexts = processed_df.iloc[indices[0]]['chunk'].tolist()

# print('relevent context : ',relevant_contexts)
context = relevant_contexts[0]

# relevant_contexts = "Brand Name: Bismuth Stibium, Generic Name: BISMUTH STIBIUM, Manufacturer Name: Uriel Pharmacy Inc."
# query = "What is Bismuth Stibium ?"

# For GPT-2
generated_answer = generate_answer_with_context_gpt2(query, context)
print("GPT-2 model")
print(generated_answer)
print("------------------------")

# For T5
generated_answer = generate_answer_with_context_t5(query, context)
print("T5 model")
print(generated_answer)
print("------------------------")
# For BERT
generated_answer = generate_answer_with_context_bert(query, context)
print("BERT model")
print(generated_answer)
print("------------------------")
# For DistilBERT
generated_answer = generate_answer_with_context_distilbert(query, context)
print("distilbert model")
print(generated_answer)
print("------------------------")

# For BLOOM
generated_answer = generate_answer_with_context_bloom(query, context)
print("bloom model")
print(generated_answer)
print("------------------------")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

bloom model
What is Bismuth Stibium ?
Answer: Bismuth Stibium is a natural product derived from the roots of the herb Bismuth. It is a natural product derived from the roots of the herb Bismuth. It is a natural product derived from the roots of the herb Bismuth. It is a natural product derived from the roots of the herb Bismuth. It is a natural product derived from the roots of the herb Bismuth. It is a natural product derived from the roots of the herb Bismuth. It is a natural product derived
------------------------


In [46]:
# from rouge_score import rouge_scorer
# from nltk.translate.bleu_score import sentence_bleu
# from sklearn.metrics import precision_recall_fscore_support

# def evaluate_answers(predicted_answers, reference_answers):
#     # Initialize ROUGE scorer
#     scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    
#     results = {
#         'rouge_scores': [],
#         'bleu_scores': [],
#         'exact_match': [],
#         'f1_scores': []
#     }
    
#     for pred, ref in zip(predicted_answers, reference_answers):
#         # ROUGE scores
#         rouge_scores = scorer.score(pred, ref)
#         results['rouge_scores'].append(rouge_scores)
        
#         # BLEU score
#         ref_tokens = [ref.split()]
#         pred_tokens = pred.split()
#         bleu = sentence_bleu(ref_tokens, pred_tokens)
#         results['bleu_scores'].append(bleu)
        
#         # Exact Match
#         exact_match = 1 if pred.lower().strip() == ref.lower().strip() else 0
#         results['exact_match'].append(exact_match)
        
#         # F1 Score for word overlap
#         pred_words = set(pred.lower().split())
#         ref_words = set(ref.lower().split())
#         precision = len(pred_words & ref_words) / len(pred_words) if pred_words else 0
#         recall = len(pred_words & ref_words) / len(ref_words) if ref_words else 0
#         f1 = 2 * precision * recall / (precision + recall) if (precision + recall) else 0
#         results['f1_scores'].append(f1)
    
#     return results

# def calculate_average_metrics(results):
#     avg_metrics = {
#         'avg_rouge1': np.mean([s['rouge1'].fmeasure for s in results['rouge_scores']]),
#         'avg_rouge2': np.mean([s['rouge2'].fmeasure for s in results['rouge_scores']]),
#         'avg_rougeL': np.mean([s['rougeL'].fmeasure for s in results['rouge_scores']]),
#         'avg_bleu': np.mean(results['bleu_scores']),
#         'avg_exact_match': np.mean(results['exact_match']),
#         'avg_f1': np.mean(results['f1_scores'])
#     }
#     return avg_metrics

In [50]:
# test_data = [
#     {
#         'question': 'What is Bismuth Stibium?',
#         'context': context,  # Your retrieved context
#         'reference_answer': 'Bismuth Stibium is a human over-the-counter (OTC) drug manufactured by Uriel Pharmacy Inc., used topically for the temporary relief of warts. It contains a blend of active ingredients, including Allium sativa (Garlic), Chelidonium (Greater celandine), Curcuma (Turmeric), Thuja (American arborvitae), Bismuth, and Stibium (Antimony). The product is for external use only and is marketed as a homeopathic remedy based on traditional practices, though it is not FDA evaluated. The ointment is applied once or twice daily to warts, with special instructions to consult a doctor for children under age 2.'  # Ground truth answer
#     },
#     # Add more test cases
# ]

# # Evaluate each model
# models = {
#     'GPT-2': generate_answer_with_context_gpt2,
#     'T5': generate_answer_with_context_t5,
#     'BERT': generate_answer_with_context_bert,
#     'DistilBERT': generate_answer_with_context_distilbert,
#     'BLOOM': generate_answer_with_context_bloom
# }

# def evaluate_all_models(test_data, models):
#     results = {}
    
#     for model_name, model_func in models.items():
#         predicted_answers = []
#         reference_answers = []
        
#         for test_case in test_data:
#             pred_answer = model_func(test_case['question'], test_case['context'])
#             predicted_answers.append(pred_answer)
#             reference_answers.append(test_case['reference_answer'])
        
#         # Evaluate answers
#         eval_results = evaluate_answers(predicted_answers, reference_answers)
#         avg_metrics = calculate_average_metrics(eval_results)
#         results[model_name] = avg_metrics
#     print(results)
#     return results

# # Run evaluation
# evaluation_results = evaluate_all_models(test_data, models)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'GPT-2': {'avg_rouge1': 0.28723404255319157, 'avg_rouge2': 0.053763440860215055, 'avg_rougeL': 0.1595744680851064, 'avg_bleu': 0.32030896835479866, 'avg_exact_match': 0.0, 'avg_f1': 0.20437956204379565}, 'T5': {'avg_rouge1': 0.2835820895522388, 'avg_rouge2': 0.10606060606060606, 'avg_rougeL': 0.17910447761194026, 'avg_bleu': 0.0392018122070561, 'avg_exact_match': 0.0, 'avg_f1': 0.16822429906542055}, 'BERT': {'avg_rouge1': 0.0, 'avg_rouge2': 0.0, 'avg_rougeL': 0.0, 'avg_bleu': 0.0, 'avg_exact_match': 0.0, 'avg_f1': 0.0}, 'DistilBERT': {'avg_rouge1': 0.0, 'avg_rouge2': 0.0, 'avg_rougeL': 0.0, 'avg_bleu': 0.0, 'avg_exact_match': 0.0, 'avg_f1': 0.0}, 'BLOOM': {'avg_rouge1': 0.24175824175824176, 'avg_rouge2': 0.044444444444444446, 'avg_rougeL': 0.2087912087912088, 'avg_bleu': 0.035371965931008, 'avg_exact_match': 0.0, 'avg_f1': 0.17582417582417584}}


In [51]:
# import time
# import psutil
# import torch

# def measure_performance(func, *args):
#     # Start measuring time and memory
#     start_time = time.time()
#     start_memory = psutil.Process().memory_info().rss / 1024 / 1024  # Memory in MB
    
#     # Run the function
#     if torch.cuda.is_available():
#         torch.cuda.reset_peak_memory_stats()
    
#     result = func(*args)
    
#     # Calculate metrics
#     end_time = time.time()
#     end_memory = psutil.Process().memory_info().rss / 1024 / 1024
    
#     metrics = {
#         'execution_time': end_time - start_time,
#         'memory_usage': end_memory - start_memory,
#     }
    
#     if torch.cuda.is_available():
#         metrics['gpu_memory'] = torch.cuda.max_memory_allocated() / 1024 / 1024  # MB
    
#     return result, metrics

In [52]:
# def create_evaluation_report(evaluation_results, performance_metrics):
#     import pandas as pd
    
#     # Create DataFrame for quality metrics
#     quality_df = pd.DataFrame(evaluation_results).round(4)
    
#     # Create DataFrame for performance metrics
#     performance_df = pd.DataFrame(performance_metrics).round(4)
    
#     # Generate report
#     report = f"""RAG System Evaluation Report
    
# Quality Metrics:
# {quality_df.to_string()}

# Performance Metrics:
# {performance_df.to_string()}

# Summary:
# - Best model for ROUGE-1: {quality_df.loc['avg_rouge1'].idxmax()} ({quality_df.loc['avg_rouge1'].max():.4f})
# - Best model for F1: {quality_df.loc['avg_f1'].idxmax()} ({quality_df.loc['avg_f1'].max():.4f})
# - Fastest model: {performance_df.loc['execution_time'].idxmin()} ({performance_df.loc['execution_time'].min():.4f}s)
# """
#     return report

In [15]:
import numpy as np
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
import time
import torch
import pandas as pd
import nltk
nltk.download('punkt')  # Required for BLEU score

# First, let's define our test dataset with some sample questions and reference answers
test_data = [
    {
        'question': 'What is Bismuth Stibium?',
        'context': context,  # Your existing context
        'reference_answer': 'Bismuth Stibium is a human over-the-counter (OTC) drug manufactured by Uriel Pharmacy Inc., used topically for the temporary relief of warts. It contains a blend of active ingredients, including Allium sativa (Garlic), Chelidonium (Greater celandine), Curcuma (Turmeric), Thuja (American arborvitae), Bismuth, and Stibium (Antimony). The product is for external use only and is marketed as a homeopathic remedy based on traditional practices, though it is not FDA evaluated. The ointment is applied once or twice daily to warts, with special instructions to consult a doctor for children under age 2.'
    },
    # Add more test cases if you have them
]

# Keep your existing model functions
models = {
    'GPT-2': generate_answer_with_context_gpt2,
    'T5': generate_answer_with_context_t5,
    'BERT': generate_answer_with_context_bert,
    'DistilBERT': generate_answer_with_context_distilbert,
    'BLOOM': generate_answer_with_context_bloom
}

def evaluate_single_answer(predicted, reference):
    # Initialize ROUGE scorer
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    
    # ROUGE scores
    rouge_scores = scorer.score(predicted, reference)
    
    # BLEU score
    ref_tokens = [reference.split()]
    pred_tokens = predicted.split()
    try:
        bleu = sentence_bleu(ref_tokens, pred_tokens)
    except:
        bleu = 0
    
    # Exact Match
    exact_match = 1 if predicted.lower().strip() == reference.lower().strip() else 0
    
    # F1 Score
    pred_words = set(predicted.lower().split())
    ref_words = set(reference.lower().split())
    
    precision = len(pred_words & ref_words) / len(pred_words) if pred_words else 0
    recall = len(pred_words & ref_words) / len(ref_words) if ref_words else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) else 0
    
    return {
        'rouge1': rouge_scores['rouge1'].fmeasure,
        'rouge2': rouge_scores['rouge2'].fmeasure,
        'rougeL': rouge_scores['rougeL'].fmeasure,
        'bleu': bleu,
        'exact_match': exact_match,
        'f1': f1
    }

def evaluate_model(model_name, model_func, test_data):
    all_metrics = []
    execution_times = []
    
    for test_case in test_data:
        # Measure execution time
        start_time = time.time()
        predicted_answer = model_func(test_case['question'], test_case['context'])
        execution_time = time.time() - start_time
        
        # Calculate metrics
        metrics = evaluate_single_answer(predicted_answer, test_case['reference_answer'])
        metrics['execution_time'] = execution_time
        all_metrics.append(metrics)
        
        # Print individual results
        print(f"\n{model_name} Results for Question: {test_case['question']}")
        print(f"Predicted Answer: {predicted_answer}")
        print(f"Reference Answer: {test_case['reference_answer']}")
        print(f"Metrics: {metrics}")
    
    # Calculate averages
    avg_metrics = {
        metric: np.mean([m[metric] for m in all_metrics])
        for metric in all_metrics[0].keys()
    }
    
    return avg_metrics

# Run evaluation for all models
results = {}
for model_name, model_func in models.items():
    print(f"\nEvaluating {model_name}...")
    try:
        results[model_name] = evaluate_model(model_name, model_func, test_data)
    except Exception as e:
        print(f"Error evaluating {model_name}: {str(e)}")
        continue

# Create final report
df_results = pd.DataFrame(results).round(4)
print("\nFinal Evaluation Results:")
print(df_results)

# Save results to CSV
df_results.to_csv('rag_evaluation_results.csv')
print("\nResults saved to 'rag_evaluation_results.csv'")

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!

Evaluating GPT-2...


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)



GPT-2 Results for Question: What is Bismuth Stibium?
Predicted Answer: This product is a topical product that is used to treat wart, ulcers, and other skin conditions. It is also used for topical use to prevent or treat other conditions such as acne, psoriasis, eczema, rashes, dry skin, skin cancer, etc.
What is the difference between Bistuth and Bactulose? BISTUTH is an antiseptic that has been used in the treatment of psoriatic ulcerative colitis, acne vulgaris, dermatitis and psorsitis. BACTULOSE is another topical antisera that was used as a treatment for psorectal ulicosis. The difference is that BIS is not a preservative. This is because B
Reference Answer: Bismuth Stibium is a human over-the-counter (OTC) drug manufactured by Uriel Pharmacy Inc., used topically for the temporary relief of warts. It contains a blend of active ingredients, including Allium sativa (Garlic), Chelidonium (Greater celandine), Curcuma (Turmeric), Thuja (American arborvitae), Bismuth, and Stibium (Anti

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)



T5 Results for Question: What is Bismuth Stibium?
Predicted Answer: : Bismuth Stibium, Generic Name: BISMUTH STIBIUM, Manufacturer Name: Uriel Pharmacy Inc., Product Type: HUMAN OTC DRUG, Route: TOPICAL, Effective Time: 20240122, Effective Date: 2024-01-22, Active Ingredient: Active Ingredients: 100 gm contains: 25 gm Allium sativa (Garlic) 1X, 25 gm Chelid
Reference Answer: Bismuth Stibium is a human over-the-counter (OTC) drug manufactured by Uriel Pharmacy Inc., used topically for the temporary relief of warts. It contains a blend of active ingredients, including Allium sativa (Garlic), Chelidonium (Greater celandine), Curcuma (Turmeric), Thuja (American arborvitae), Bismuth, and Stibium (Antimony). The product is for external use only and is marketed as a homeopathic remedy based on traditional practices, though it is not FDA evaluated. The ointment is applied once or twice daily to warts, with special instructions to consult a doctor for children under age 2.
Metrics: {'rouge1': 

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



BERT Results for Question: What is Bismuth Stibium?
Predicted Answer: 
Reference Answer: Bismuth Stibium is a human over-the-counter (OTC) drug manufactured by Uriel Pharmacy Inc., used topically for the temporary relief of warts. It contains a blend of active ingredients, including Allium sativa (Garlic), Chelidonium (Greater celandine), Curcuma (Turmeric), Thuja (American arborvitae), Bismuth, and Stibium (Antimony). The product is for external use only and is marketed as a homeopathic remedy based on traditional practices, though it is not FDA evaluated. The ointment is applied once or twice daily to warts, with special instructions to consult a doctor for children under age 2.
Metrics: {'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0, 'bleu': 0, 'exact_match': 0, 'f1': 0, 'execution_time': 4.387006521224976}

Evaluating DistilBERT...

DistilBERT Results for Question: What is Bismuth Stibium?
Predicted Answer: 
Reference Answer: Bismuth Stibium is a human over-the-counter (OTC) drug manu

In [37]:
# from transformers import BertForQuestionAnswering, BertTokenizer
# import torch

# def generate_answer_with_context_bert(query, context):
#     model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
#     model = BertForQuestionAnswering.from_pretrained(model_name)
#     tokenizer = BertTokenizer.from_pretrained(model_name)

#     # Prepare the input text for BERT
#     input_text = f"Context: {context}\nQuestion: {query}"
#     inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    
#     # Run the model to get start and end logits
#     outputs = model(**inputs)
#     start_scores, end_scores = outputs.start_logits, outputs.end_logits

#     # Debug: Print the logits to understand where the model is looking for the answer
#     # print("Start Scores:", start_scores)
#     # print("End Scores:", end_scores)

#     # Get the most likely start and end positions
#     answer_start = torch.argmax(start_scores)
#     answer_end = torch.argmax(end_scores) + 1

#     # Check if the answer span is valid (ensuring answer_start < answer_end)
#     if answer_start >= answer_end:
#         print("No valid answer found.")
#         return "No valid answer found."

#     # Decode the answer (removes [CLS], [SEP] tokens and gives you the answer span)
#     answer = tokenizer.decode(inputs.input_ids[0][answer_start:answer_end], skip_special_tokens=True)
    
#     # Check if the answer is meaningful or just a part of the question
#     if answer.lower() == query.lower():
#         print("The model returned the question itself, no valid answer found.")
#         return "No valid answer found."

#     return answer

# # Example of running the function with different inputs
# query = "What is the dosage for the drug?"
# context = "Brand Name: Bismuth Stibium, Generic Name: BISMUTH STIBIUM, Manufacturer Name: Uriel Pharmacy Inc., Directions: Apply once or twice daily to warts."
# generated_answer = generate_answer_with_context_bert(query, context)

# print("BERT model")
# print(generated_answer)
# print("------------------------")


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERT model
dosage
------------------------
